# 第一章、知识库文档处理

 - [一、 文档加载](#一、-文档加载)
     - [1.1 探索加载的数据](#1.1-探索加载的数据)
 - [二、文档分割](#二、文档分割)
 - [三、文档词向量化](#三、文档词向量化)


## 一、 文档加载

我们使用 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

下载文档到`../../docs/knowledge_base/`目录下(可选，也替换为自己的文档)：
- [《李宏毅深度学习教程》](https://github.com/datawhalechina/leedl-tutorial/releases/download/v1.1.1/LeeDL_Tutorial_v.1.1.1.pdf)
- [《蘑菇书EasyRL》](https://github.com/datawhalechina/easy-rl/releases/download/v1.0.6/EasyRL_v1.0.6.pdf)
- [《大语言模型中文综述》](https://github.com/RUCAIBox/LLMSurvey/blob/main/assets/LLM_Survey__Chinese_V1.pdf)


In [1]:
## 安装必要的库
# !pip install rapidocr_onnxruntime -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install "unstructured[all-docs]" -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install pyMuPDF -i https://pypi.tuna.tsinghua.edu.cn/simple

In [7]:
from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../../docs/knowledge_base/LeeDL_Tutorial.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()

### 1.1 探索加载的数据

文档加载后储存在 `pages` 变量中:
- `page` 的变量类型为 `List`
- 打印 `pages` 的长度可以看到 pdf 一共包含多少页

In [8]:
print(f"载入后的变量类型为：{type(pages)}，",  f"该 PDF 一共包含 {len(pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 330 页


`page` 中的每一元素为一个文档，变量类型为 `langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [15]:
page = pages[1]
print(f"每一个元素的类型：{type(page)}.", 
    f"该文档的描述性数据：{page.metadata}", 
    f"查看该文档的内容:\n{page.page_content[0:1000]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain.schema.document.Document'>.
------
该文档的描述性数据：{'source': '../../docs/knowledge_base/LeeDL_Tutorial.pdf', 'file_path': '../../docs/knowledge_base/LeeDL_Tutorial.pdf', 'page': 1, 'total_pages': 330, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230831225119+08'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
前言
李宏毅老师是台湾大学的教授，其《机器学习》（2021 年春）是深度学习领域经典的中文视
频之一。李老师幽默风趣的授课风格深受大家喜爱，让晦涩难懂的深度学习理论变得轻松易懂，
他会通过很多动漫相关的有趣例子来讲解深度学习理论。李老师的课程内容很全面，覆盖了到深
度学习必须掌握的常见理论，能让学生对于深度学习的绝大多数领域都有一定了解，从而可以进
一步选择想要深入的方向进行学习，对于想入门深度学习又想看中文讲解的同学是非常推荐的。
本教程主要内容源于《机器学习》（2021 年春），并在其基础上进行了一定的原创。比如，为
了尽可能地降低阅读门槛，笔者对这门公开课的精华内容进行选取并优化，对所涉及的公式都给
出详细的推导过程，对较难理解的知识点进行了重点讲解和强化，以方便读者较为轻松地入门。
此外，为了丰富内容，笔者在教程中选取了《机器学习》（2017 年春） 的部分内容，并补充了不
少除这门公开课之外的深度学习相关知识。
致谢
特别感谢 Sm1les、LSGOMYP 对本项目的帮助与支持。
扫描下方二维码，然后回复关键词“李宏毅深度学习”，即可加入“LeeDL-Tutorial 读者交流群”
Datawhale
一个专注于 AI 领域的开源组织
版权声明
本作

可以看到我们成功导入了我们所需的文档。接下来我们将文档进行切分

## 二、文档分割
Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../../figures/example-splitter.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [16]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [28]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [29]:
# 使用递归字符文本分割器
from langchain.text_splitter import TokenTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(page.page_content[0:1000])

['前言\n李宏毅老师是台湾大学的教授，其《机器学习》（2021 年春）是深度学习领域经典的中文视\n频之一。李老师幽默风趣的授课风格深受大家喜爱，让晦涩难懂的深度学习理论变得轻松易懂，\n他会通过很多动漫相关的有趣例子来讲解深度学习理论。李老师的课程内容很全面，覆盖了到深\n度学习必须掌握的常见理论，能让学生对于深度学习的绝大多数领域都有一定了解，从而可以进\n一步选择想要深入的方向进行学习，对于想入门深度学习又想看中文讲解的同学是非常推荐的。\n本教程主要内容源于《机器学习》（2021 年春），并在其基础上进行了一定的原创。比如，为\n了尽可能地降低阅读门槛，笔者对这门公开课的精华内容进行选取并优化，对所涉及的公式都给\n出详细的推导过程，对较难理解的知识点进行了重点讲解和强化，以方便读者较为轻松地入门。\n此外，为了丰富内容，笔者在教程中选取了《机器学习》（2017 年春） 的部分内容，并补充了不\n少除这门公开课之外的深度学习相关知识。\n致谢\n特别感谢 Sm1les、LSGOMYP 对本项目的帮助与支持。',
 '致谢\n特别感谢 Sm1les、LSGOMYP 对本项目的帮助与支持。\n扫描下方二维码，然后回复关键词“李宏毅深度学习”，即可加入“LeeDL-Tutorial 读者交流群”\nDatawhale\n一个专注于 AI 领域的开源组织\n版权声明\n本作品采用知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议进行许可。']

In [31]:
split_docs = text_splitter.split_documents(pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：846


In [32]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：338773


## 三、文档词向量化

在机器学习和自然语言处理（NLP）中，`Embeddings`（嵌入）是一种将类别数据，如单词、句子或者整个文档，转化为实数向量的技术。这些实数向量可以被计算机更好地理解和处理。嵌入背后的主要想法是，相似或相关的对象在嵌入空间中的距离应该很近。

举个例子，我们可以使用词嵌入（word embeddings）来表示文本数据。在词嵌入中，每个单词被转换为一个向量，这个向量捕获了这个单词的语义信息。例如，"king" 和 "queen" 这两个单词在嵌入空间中的位置将会非常接近，因为它们的含义相似。而 "apple" 和 "orange" 也会很接近，因为它们都是水果。而 "king" 和 "apple" 这两个单词在嵌入空间中的距离就会比较远，因为它们的含义不同。

让我们取出我们的切分部分并对它们进行 `Embedding` 处理。

这里提供三种方式进行，一种是直接使用 openai 的模型去生成 embedding，另一种是使用 HuggingFace 上的模型去生成 embedding。

- openAI 的模型需要消耗 api，对于大量的token 来说成本会比较高，但是非常方便。
- HuggingFace 的模型可以本地部署，可自定义合适的模型，可玩性较高，但对本地的资源有部分要求。
- 采用其他平台的 api。对于获取 openAI key 不方便的同学可以采用这种方法。

对于只想体验一下的同学来说，可以尝试直接用生成好的 embedding，或者在本地部署小模型进行尝试。

HuggingFace 是一个优秀的开源库，我们只需要输入模型的名字，就会自动帮我们解析对应的能力。

In [ ]:
# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env fileopenai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key  = os.environ['OPENAI_API_KEY']
zhihuai.api_key = os.environ['ZHIPUAI_API_KEY']

In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

# embedding = OpenAIEmbeddings() 
# embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
embedding = ZhipuAIEmbeddings()

In [38]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
query1 = "机器学习"
query2 = "强化学习"
query3 = "大语言模型"

# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = embedding.embed_query(query1)
emb2 = embedding.embed_query(query2)
emb3 = embedding.embed_query(query3)

# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)

可以直接查看 embedding 的具体信息，embedding 的维度通常取决于所使用的模型。

In [42]:
print(f"{query1} 生成的为长度 {len(emb1)} 的 embedding , 其前 30 个值为： {emb1[:30]}") 

机器学习 生成的为长度 1024 的 embedding , 其前 30 个值为： [-0.02768379  0.07836673  0.1429528  -0.1584693   0.08204    -0.15819356
 -0.01282174  0.18076552  0.20916627  0.21330206 -0.1205181  -0.06666514
 -0.16731478  0.31798768  0.0680017  -0.13807729 -0.03469152  0.15737721
  0.02108428 -0.29145902 -0.10099868  0.20487919 -0.03603597 -0.09646764
  0.12923686 -0.20558454  0.17238656  0.03429411  0.1497675  -0.25297147]


我们已经生成了对应的向量，我们如何度量文档和问题的相关性呢？

这里提供两种常用的方法：
- 计算两个向量之间的点积。
- 计算两个向量之间的余弦相似度

点积是将两个向量对应位置的元素相乘后求和得到的标量值。点积相似度越大，表示两个向量越相似。

这里直接使用 numpy 的函数进行计算

In [43]:
print(f"{query1} 和 {query2} 向量之间的点积为：{np.dot(emb1, emb2)}")
print(f"{query1} 和 {query3} 向量之间的点积为：{np.dot(emb1, emb3)}")
print(f"{query2} 和 {query3} 向量之间的点积为：{np.dot(emb2, emb3)}")

机器学习 和 强化学习 向量之间的点积为：17.218882120572722
机器学习 和 大语言模型 向量之间的点积为：16.522186236712727
强化学习 和 大语言模型 向量之间的点积为：11.368461841901752


点积：计算简单，快速，不需要进行额外的归一化步骤，但丢失了方向信息。

余弦相似度：可以同时比较向量的方向和数量级大小。

余弦相似度将两个向量的点积除以它们的模长的乘积。其基本的计算公式为 $$cos(\theta) = {\sum_{i=1}^{n}{(x_i \times y_i)} \over {\sum_{i=1}^{n}{(x_i)^2} \times \sum_{i=1}^{n}{(y_i)^2}}}$$余弦函数的值域在-1到1之间，即两个向量余弦相似度的范围是[-1, 1]。当两个向量夹角为0°时，即两个向量重合时，相似度为1；当夹角为180°时，即两个向量方向相反时，相似度为-1。即越接近于 1 越相似，越接近 0 越不相似。

In [44]:
print(f"{query1} 和 {query2} 向量之间的余弦相似度为：{cosine_similarity(emb1.reshape(1, -1) , emb2.reshape(1, -1) )}")
print(f"{query1} 和 {query3} 向量之间的余弦相似度为：{cosine_similarity(emb1.reshape(1, -1) , emb3.reshape(1, -1) )}")
print(f"{query2} 和 {query3} 向量之间的余弦相似度为：{cosine_similarity(emb2.reshape(1, -1) , emb3.reshape(1, -1) )}")

机器学习 和 强化学习 向量之间的余弦相似度为：[[0.68814796]]
机器学习 和 大语言模型 向量之间的余弦相似度为：[[0.63382724]]
强化学习 和 大语言模型 向量之间的余弦相似度为：[[0.43555894]]


可以看出，模型认为机器学习和强化学习更相关一点，强化学习和大语言模型之间的相关性更差。（这部分跟训练语料的时间相关，embedding 的模型应该没有大语言模型相关的语料。）

目前，我们已经学习了文档的基本处理，但是如何管理我们生成的 embedding 并寻找和 query 最相关的内容呢？难道要每次遍历所有文档么？向量数据库可以帮我们快速的管理和计算这些内容。